In [1]:

import numpy as np 


In [2]:
!git clone 

Cloning into 'FaceLandmark_PFLD_UltraLight'...
remote: Enumerating objects: 198, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 198 (delta 13), reused 0 (delta 0), pack-reused 174
Receiving objects: 100% (198/198), 19.18 MiB | 17.26 MiB/s, done.
Resolving deltas: 100% (90/90), done.


In [3]:
cd /kaggle/working/FaceLandmark_PFLD_UltraLight

/kaggle/working/FaceLandmark_PFLD_UltraLight


In [ ]:
!pip install -r requirement.txt

In [5]:
!cp -r /kaggle/input/wider-facial-landmarks-inthewild-wflw/WFLW ./data

In [6]:
cd data

/kaggle/working/FaceLandmark_PFLD_UltraLight/data


In [8]:
cd /kaggle/working/FaceLandmark_PFLD_UltraLight

/kaggle/working/FaceLandmark_PFLD_UltraLight


In [12]:
!python train.py

  File "train.py", line 131
    logging.warning('=======>>>>>>> Scheduler Generated' + '
                                                           ^
SyntaxError: EOL while scanning string literal
